# Charts by store

In [ ]:
import os
import numpy
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from itertools import cycle
import statsmodels.api as sm 
from scipy.interpolate import interp1d
import datetime as dt

pd.set_option('max_columns', 50)
plt.style.use('bmh')
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

In [ ]:
osj = os.path.join
INPUT_DIR = '../input/m5-forecasting-accuracy/'
train = pd.read_csv(osj(INPUT_DIR, 'sales_train_validation.csv'))
price = pd.read_csv(osj(INPUT_DIR, 'sell_prices.csv'))
calender = pd.read_csv(osj(INPUT_DIR, 'calendar.csv'))

sample_submit = pd.read_csv(osj(INPUT_DIR, 'sample_submission.csv'))

In [ ]:
calender['date'] = pd.to_datetime(calender['date'])

In [ ]:
d_cols = [c for c in train.columns if 'd_' in c] # d_で始まる日付列のリスト

In [ ]:
past_sales = train.set_index('id')[d_cols].T.merge(calender.set_index('d')['date'],
                                                   left_index=True,
                                                   right_index=True,
                                                   validate='1:1').set_index('date')

In [ ]:
def make_total_sales_lowess( p ):
    total_sales = pd.DataFrame(p.sum(axis=1), columns=['total sales'])
    
    # クリスマス削除
    total_sales_noXmas = total_sales.drop(index=[dt.datetime(2011,12,25), 
                                                 dt.datetime(2012,12,25), 
                                                 dt.datetime(2013,12,25), 
                                                 dt.datetime(2014,12,25), 
                                                 dt.datetime(2015,12,25)])

    df = total_sales_noXmas
    lowess = sm.nonparametric.lowess(df['total sales'], df.index, frac=.3) 
    lowess_x = list(zip(*lowess))[0] 
    lowess_y = list(zip(*lowess))[1] 
    
    f = interp1d(lowess_x, lowess_y, bounds_error=False)
    new_lowess_x = df.index
    new_lowess_y = f(new_lowess_x)
    
    total_sales_lowess = total_sales_noXmas
    total_sales_lowess['lowess'] = new_lowess_y
    total_sales_lowess['total sales-lowess'] = total_sales_lowess['total sales'] - total_sales_lowess['lowess']
    
    return total_sales_lowess

In [ ]:
total_sales_lowess = make_total_sales_lowess( past_sales )

In [ ]:
y_bt = -5000
y_up = 10000
store_list = price['store_id'].unique()
for store in store_list :
    total_sales_lowess_tmp = make_total_sales_lowess( past_sales.loc[:, past_sales.columns.str.contains(store)] )
    total_sales_lowess_tmp.plot(figsize=(15, 5), alpha=0.8, ylim=[y_bt, y_up],title=store)

# Charts trends

We can found following chart trends in recently:

CA_1 : small -

CA_2 : +

CA_3 : small -

CA_4 : small +

TX_1 : stable

TX_2 : small +

TX_3 : small +

WI_1 : +

WI_2 : ++

WI_3 : +

Let's adjust!

# Adjust submission

In [ ]:
########################### Read base submission
############################################################################
submission = pd.read_csv('../input/m5-three-shades-of-dark-darker-magic/submission_v1.csv')

In [ ]:
########################### Day 1
############################################################################

# Current number of teams       3,215 (day 2 of May)
# Number of teams 2 days before 3,186 
teams_now = 3215
teams_before = 3186
#submission['F1'] *= teams_now/teams_before
submission.loc[submission['id'].str.contains('CA_1'), 'F1'] *= 1.009
submission.loc[submission['id'].str.contains('CA_2'), 'F1'] *= 1.02
submission.loc[submission['id'].str.contains('CA_3'), 'F1'] *= 1.009
submission.loc[submission['id'].str.contains('CA_4'), 'F1'] *= 1.011

submission.loc[submission['id'].str.contains('TX_1'), 'F1'] *= 1.01
submission.loc[submission['id'].str.contains('TX_2'), 'F1'] *= 1.011
submission.loc[submission['id'].str.contains('TX_3'), 'F1'] *= 1.011

submission.loc[submission['id'].str.contains('WI_1'), 'F1'] *= 1.015
submission.loc[submission['id'].str.contains('WI_2'), 'F1'] *= 1.02
submission.loc[submission['id'].str.contains('WI_3'), 'F1'] *= 1.015
submission['F1'].sum()

In [ ]:
########################### Day 2
############################################################################

# In case you'll get a heart attack 
# because of all the MAGIC in this kernel -
# here is South Korean emergency number - 119
magic = submission['F2'].sum()
japan_emergency = 119
#submission['F2'] *= magic / (magic + japan_emergency)
                             
submission.loc[submission['id'].str.contains('CA_1'), 'F2'] *= 0.994
submission.loc[submission['id'].str.contains('CA_2'), 'F2'] *= 0.998
submission.loc[submission['id'].str.contains('CA_3'), 'F2'] *= 0.994
submission.loc[submission['id'].str.contains('CA_4'), 'F2'] *= 0.996

submission.loc[submission['id'].str.contains('TX_1'), 'F2'] *= 0.995
submission.loc[submission['id'].str.contains('TX_2'), 'F2'] *= 0.996
submission.loc[submission['id'].str.contains('TX_3'), 'F2'] *= 0.996

submission.loc[submission['id'].str.contains('WI_1'), 'F2'] *= 0.998
submission.loc[submission['id'].str.contains('WI_2'), 'F2'] *= 1
submission.loc[submission['id'].str.contains('WI_3'), 'F2'] *= 0.998

submission['F2'].sum()

In [ ]:
########################### Day 3
############################################################################

# 1913 - The Woman suffrage parade of 1913 takes place in Washington, D.C.
# 1923 - Ankara replaces Istanbul (Constantinople), as the capital of Turkey.
#submission['F3'] *= 1913/1923
submission.loc[submission['id'].str.contains('CA_1'), 'F3'] *= 0.994
submission.loc[submission['id'].str.contains('CA_2'), 'F3'] *= 0.998
submission.loc[submission['id'].str.contains('CA_3'), 'F3'] *= 0.994
submission.loc[submission['id'].str.contains('CA_4'), 'F3'] *= 0.996

submission.loc[submission['id'].str.contains('TX_1'), 'F3'] *= 0.995
submission.loc[submission['id'].str.contains('TX_2'), 'F3'] *= 0.996
submission.loc[submission['id'].str.contains('TX_3'), 'F3'] *= 0.996

submission.loc[submission['id'].str.contains('WI_1'), 'F3'] *= 1
submission.loc[submission['id'].str.contains('WI_2'), 'F3'] *= 1.005
submission.loc[submission['id'].str.contains('WI_3'), 'F3'] *= 1
submission['F3'].sum()

In [ ]:
########################### Day 4
############################################################################

# I love statistics.
# When you dig in data you can 
# find VERY interesting "facts".
# Did you know that the mean 
# number of sexual partners 
# per person is 1.0073
sex_constant = 1.0073
#submission['F4'] /= sex_constant
submission.loc[submission['id'].str.contains('CA_1'), 'F4'] *= 0.98
submission.loc[submission['id'].str.contains('CA_2'), 'F4'] *= 1.02
submission.loc[submission['id'].str.contains('CA_3'), 'F4'] *= 0.98
submission.loc[submission['id'].str.contains('CA_4'), 'F4'] *= 1

submission.loc[submission['id'].str.contains('TX_1'), 'F4'] *= 0.99
submission.loc[submission['id'].str.contains('TX_2'), 'F4'] *= 1
submission.loc[submission['id'].str.contains('TX_3'), 'F4'] *= 1

submission.loc[submission['id'].str.contains('WI_1'), 'F4'] *= 1.00
submission.loc[submission['id'].str.contains('WI_2'), 'F4'] *= 1.02
submission.loc[submission['id'].str.contains('WI_3'), 'F4'] *= 1.00
submission['F4'].sum()

In [ ]:
########################### Day 5
############################################################################

# It's Friday  
# Friday night is a mix (colors)
# of girls #ffb0e8 and boys #330000
# Resulted mix HEX color code is #995874
color_of_the_night = 995874

# There are millions "Stories"
# We need to make it Mean, really mean
color_of_the_night /= 1000000
#submission['F5'] *= color_of_the_night
submission.loc[submission['id'].str.contains('CA_1'), 'F5'] *= 0.995
submission.loc[submission['id'].str.contains('CA_2'), 'F5'] *= 1
submission.loc[submission['id'].str.contains('CA_3'), 'F5'] *= 0.995
submission.loc[submission['id'].str.contains('CA_4'), 'F5'] *= 0.998

submission.loc[submission['id'].str.contains('TX_1'), 'F5'] *= 0.996
submission.loc[submission['id'].str.contains('TX_2'), 'F5'] *= 0.998
submission.loc[submission['id'].str.contains('TX_3'), 'F5'] *= 0.998

submission.loc[submission['id'].str.contains('WI_1'), 'F5'] *= 1
submission.loc[submission['id'].str.contains('WI_2'), 'F5'] *= 1.01
submission.loc[submission['id'].str.contains('WI_3'), 'F5'] *= 1
submission['F5'].sum()

In [ ]:
########################### Day 6
############################################################################

# Fridays not always pass without "consequences"
# BE CAREFUL - DRINK MODERATELY
# https://www.ncbi.nlm.nih.gov/pubmed/25701909
# Traumatic brain injury and cognition.
# PMID: 25701909 DOI: 10.1016/B978-0-444-63521-1.00037-6

# 1.00037-6 - it can't be just a coincidence
injury = 1.000376
#submission['F6'] *= injury
submission.loc[submission['id'].str.contains('CA_1'), 'F6'] *= 1.0002
submission.loc[submission['id'].str.contains('CA_2'), 'F6'] *= 1.001
submission.loc[submission['id'].str.contains('CA_3'), 'F6'] *= 1.0002
submission.loc[submission['id'].str.contains('CA_4'), 'F6'] *= 1.001

submission.loc[submission['id'].str.contains('TX_1'), 'F6'] *= 1.000
submission.loc[submission['id'].str.contains('TX_2'), 'F6'] *= 1.001
submission.loc[submission['id'].str.contains('TX_3'), 'F6'] *= 1.001

submission.loc[submission['id'].str.contains('WI_1'), 'F6'] *= 1.001
submission.loc[submission['id'].str.contains('WI_2'), 'F6'] *= 1.01
submission.loc[submission['id'].str.contains('WI_3'), 'F6'] *= 1.001
submission['F6'].sum()

In [ ]:
########################### Day 7
############################################################################

# Let's make it aaaall random
# We will need NumPy

# We will divide the COMPLETELY random number
# from range >1000000 by 1000000
# to have some float number from 0 to 1 

import numpy as np
np.random.seed(198505)
correction = np.random.randint(1000000)/1000000
#submission['F7'] *= correction
submission.loc[submission['id'].str.contains('CA_1'), 'F7'] *= 0.994
submission.loc[submission['id'].str.contains('CA_2'), 'F7'] *= 1
submission.loc[submission['id'].str.contains('CA_3'), 'F7'] *= 0.994
submission.loc[submission['id'].str.contains('CA_4'), 'F7'] *= 0.998

submission.loc[submission['id'].str.contains('TX_1'), 'F7'] *= 0.995635
submission.loc[submission['id'].str.contains('TX_2'), 'F7'] *= 0.998
submission.loc[submission['id'].str.contains('TX_3'), 'F7'] *= 0.998

submission.loc[submission['id'].str.contains('WI_1'), 'F7'] *= 1
submission.loc[submission['id'].str.contains('WI_2'), 'F7'] *= 1.01
submission.loc[submission['id'].str.contains('WI_3'), 'F7'] *= 1
submission['F7'].sum()

In [ ]:
########################### Day 8
############################################################################
#submission['F8'] *= (100-1)/100 + (100-12)/10000
submission.loc[submission['id'].str.contains('CA_1'), 'F8'] *= 0.996
submission.loc[submission['id'].str.contains('CA_2'), 'F8'] *= 1
submission.loc[submission['id'].str.contains('CA_3'), 'F8'] *= 0.996
submission.loc[submission['id'].str.contains('CA_4'), 'F8'] *= 0.998

submission.loc[submission['id'].str.contains('TX_1'), 'F8'] *= 0.9988
submission.loc[submission['id'].str.contains('TX_2'), 'F8'] *= 1
submission.loc[submission['id'].str.contains('TX_3'), 'F8'] *= 1

submission.loc[submission['id'].str.contains('WI_1'), 'F8'] *= 1
submission.loc[submission['id'].str.contains('WI_2'), 'F8'] *= 1.01
submission.loc[submission['id'].str.contains('WI_3'), 'F8'] *= 1
submission['F8'].sum()

In [ ]:
########################### Day 11
############################################################################

# Let's not change anything here
# I have a feeling that we should 
# keep origical prediction
#submission['F11'] = submission['F11']
submission.loc[submission['id'].str.contains('CA_1'), 'F11'] *= 1 
submission.loc[submission['id'].str.contains('CA_2'), 'F11'] *= 1
submission.loc[submission['id'].str.contains('CA_3'), 'F11'] *= 1
submission.loc[submission['id'].str.contains('CA_4'), 'F11'] *= 1

submission.loc[submission['id'].str.contains('TX_1'), 'F11'] *= 1
submission.loc[submission['id'].str.contains('TX_2'), 'F11'] *= 1
submission.loc[submission['id'].str.contains('TX_3'), 'F11'] *= 1

submission.loc[submission['id'].str.contains('WI_1'), 'F11'] *= 1
submission.loc[submission['id'].str.contains('WI_2'), 'F11'] *= 1
submission.loc[submission['id'].str.contains('WI_3'), 'F11'] *= 1
submission['F11'].sum()

In [ ]:
########################### Days 9-19
############################################################################

# Just 1% of his fortune is equivalent to the whole health budget for Ethiopia
for i in range(9,20):
    if i!=11:
        #submission['F'+str(i)] *= 1.01 
        submission.loc[submission['id'].str.contains('CA_1'), 'F'+str(i)] *= 1
        submission.loc[submission['id'].str.contains('CA_2'), 'F'+str(i)] *= 1.015
        submission.loc[submission['id'].str.contains('CA_3'), 'F'+str(i)] *= 1
        submission.loc[submission['id'].str.contains('CA_4'), 'F'+str(i)] *= 1.013

        submission.loc[submission['id'].str.contains('TX_1'), 'F'+str(i)] *= 1
        submission.loc[submission['id'].str.contains('TX_2'), 'F'+str(i)] *= 1.013
        submission.loc[submission['id'].str.contains('TX_3'), 'F'+str(i)] *= 1.013

        submission.loc[submission['id'].str.contains('WI_1'), 'F'+str(i)] *= 1.015
        submission.loc[submission['id'].str.contains('WI_2'), 'F'+str(i)] *= 1.02
        submission.loc[submission['id'].str.contains('WI_3'), 'F'+str(i)] *= 1.015
        print(submission['F'+str(i)].sum())

In [ ]:
########################### Days 20-28
############################################################################

# Of all the people earning A$100,000 a year under the age of 50, 2% are women. Just 2% are women.
for i in range(20,29):
    #submission['F'+str(i)] *= 1.02
    submission.loc[submission['id'].str.contains('CA_1'), 'F'+str(i)] *= 1.015
    submission.loc[submission['id'].str.contains('CA_2'), 'F'+str(i)] *= 1.03
    submission.loc[submission['id'].str.contains('CA_3'), 'F'+str(i)] *= 1.015
    submission.loc[submission['id'].str.contains('CA_4'), 'F'+str(i)] *= 1.025

    submission.loc[submission['id'].str.contains('TX_1'), 'F'+str(i)] *= 1.02
    submission.loc[submission['id'].str.contains('TX_2'), 'F'+str(i)] *= 1.025
    submission.loc[submission['id'].str.contains('TX_3'), 'F'+str(i)] *= 1.025

    submission.loc[submission['id'].str.contains('WI_1'), 'F'+str(i)] *= 1.03
    submission.loc[submission['id'].str.contains('WI_2'), 'F'+str(i)] *= 1.04
    submission.loc[submission['id'].str.contains('WI_3'), 'F'+str(i)] *= 1.03
    print(submission['F'+str(i)].sum())

In [ ]:
########################### Export
############################################################################
submission.to_csv('submission.csv', index=False)